In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Parameters
samplenum = 6
input_size = 3
output_size = 3
time_length = 3; #ms

In [3]:
# Generate simulation all p_init
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)
#sample p
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.p_init + 1
p = torch.tensor(p, requires_grad = True).t()
input = p.double()
print(p.shape)

torch.Size([6, 9])


In [4]:
# Generate simulation, random p
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
#print(y_target)
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
p = torch.tensor(p, requires_grad = True).t()
input = p.double()
print(input.shape)

torch.Size([6, 9])


## Building the custon Simulation Activation Function

In [5]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        print(f'y_pred: {y_pred}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.dtype}')
        
        grad_input = torch.tensor(grad_output.double().mm(dy_dp_batch))/samplenum
        print(f'shape of grad input: {grad_input.shape}')
        print(f'shape of grad output: {grad_output.shape}')
        print(f'grad_input: {grad_input}')
        print(f'grad_output: {grad_output}')
        return grad_input    

Simulate = Simulate.apply

In [ ]:
num = 
anal = torch.autograd.gradcheck.ge

In [6]:
#GRADCHECK
from torch.autograd import gradcheck

test = gradcheck(Simulate, (input,), eps=1e-4, atol=1e-6, raise_exception = True)
print(test)

],
        [2.1487e-05, 2.0027e+00, 1.3569e-04]])
y_pred: tensor([[2.1543e-05, 2.0002e+00, 2.5284e-05],
        [2.1511e-05, 2.0016e+00, 8.6335e-05],
        [2.1544e-05, 2.0001e+00, 2.2603e-05],
        [2.1281e-05, 2.0113e+00, 9.4354e-04],
        [2.1498e-05, 2.0022e+00, 1.1213e-04],
        [2.1487e-05, 2.0027e+00, 1.3569e-04]])
y_pred: tensor([[2.1543e-05, 2.0002e+00, 2.5284e-05],
        [2.1511e-05, 2.0016e+00, 8.6335e-05],
        [2.1545e-05, 2.0001e+00, 2.1721e-05],
        [2.1281e-05, 2.0113e+00, 9.4354e-04],
        [2.1498e-05, 2.0022e+00, 1.1213e-04],
        [2.1487e-05, 2.0027e+00, 1.3569e-04]])
y_pred: tensor([[2.1543e-05, 2.0002e+00, 2.5284e-05],
        [2.1511e-05, 2.0016e+00, 8.6335e-05],
        [2.1545e-05, 2.0001e+00, 2.1370e-05],
        [2.1281e-05, 2.0113e+00, 9.4354e-04],
        [2.1498e-05, 2.0022e+00, 1.1213e-04],
        [2.1487e-05, 2.0027e+00, 1.3569e-04]])
y_pred: tensor([[2.1543e-05, 2.0002e+00, 2.5284e-05],
        [2.1511e-05, 2.0016e+00, 8.6335e-

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 7.8794e-03,  0.0000e+00,  1.0914e-07,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.9372e-06, -5.3644e-02, -5.0389e-03,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  7.8775e-03,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 5.9196e-03,  0.0000e+00,  9.0949e-09,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00

In [32]:
#print(p.backward(torch.ones(samplenum,9)))
p.requires_grad
2.1556e-05-1.3456e-05

8.100000000000002e-06

In [7]:
from torch.autograd import gradcheck
p = torch.tensor(p, requires_grad = True).t()
input = (p.double())
test = nn.Linear(9,3)
#print(input)
test = gradcheck(test, (input,), eps=1e-3, atol=1e-3, raise_exception = True)
print(test)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm